In [7]:
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
# from nltk.corpus import stopwords 
from collections import Counter
import string
import re
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn


/tmp/ipykernel_5413/3817616041.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [8]:
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [13]:
vocab = {'br': 1, 'movie': 2, 'film': 3, 'one': 4, 'like': 5, 'good': 6, 'even': 7, 'would': 8, 'time': 9, 'really': 10, 'see': 11, 'story': 12, 'much': 13, 'well': 14, 'get': 15, 'great': 16, 'bad': 17, 'also': 18, 'people': 19, 'first': 20, 'dont': 21, 'movies': 22, 'made': 23, 'films': 24, 'make': 25, 'could': 26, 'way': 27, 'characters': 28, 'think': 29, 'watch': 30, 'many': 31, 'seen': 32, 'two': 33, 'character': 34, 'never': 35, 'acting': 36, 'love': 37, 'best': 38, 'plot': 39, 'little': 40, 'know': 41, 'show': 42, 'life': 43, 'ever': 44, 'better': 45, 'still': 46, 'say': 47, 'scene': 48, 'end': 49, 'man': 50, 'scenes': 51, 'something': 52, 'go': 53, 'im': 54, 'back': 55, 'real': 56, 'watching': 57, 'actors': 58, 'doesnt': 59, 'thing': 60, 'didnt': 61, 'years': 62, 'another': 63, 'funny': 64, 'actually': 65, 'though': 66, 'makes': 67, 'nothing': 68, 'find': 69, 'look': 70, 'going': 71, 'new': 72, 'work': 73, 'lot': 74, 'every': 75, 'old': 76, 'us': 77, 'part': 78, 'cant': 79, 'director': 80, 'thats': 81, 'want': 82, 'things': 83, 'pretty': 84, 'quite': 85, 'cast': 86, 'seems': 87, 'around': 88, 'got': 89, 'however': 90, 'young': 91, 'take': 92, 'world': 93, 'fact': 94, 'enough': 95, 'big': 96, 'ive': 97, 'thought': 98, 'give': 99, 'may': 100, 'horror': 101, 'saw': 102, 'isnt': 103, 'without': 104, 'gets': 105, 'right': 106, 'always': 107, 'long': 108, 'theres': 109, 'original': 110, 'come': 111, 'music': 112, 'almost': 113, 'comedy': 114, 'whole': 115, 'must': 116, 'role': 117, 'series': 118, 'least': 119, 'times': 120, 'interesting': 121, 'guy': 122, 'action': 123, 'done': 124, 'bit': 125, 'point': 126, 'far': 127, 'script': 128, 'might': 129, 'feel': 130, 'since': 131, 'minutes': 132, 'hes': 133, 'anything': 134, 'last': 135, 'probably': 136, 'kind': 137, 'performance': 138, 'family': 139, 'worst': 140, 'tv': 141, 'rather': 142, 'yet': 143, 'fun': 144, 'away': 145, 'played': 146, 'anyone': 147, 'sure': 148, 'found': 149, 'girl': 150, 'making': 151, 'woman': 152, 'believe': 153, 'although': 154, 'course': 155, 'trying': 156, 'shows': 157, 'hard': 158, 'especially': 159, 'put': 160, 'comes': 161, 'everything': 162, 'day': 163, 'maybe': 164, 'goes': 165, 'worth': 166, 'main': 167, 'looking': 168, 'dvd': 169, 'different': 170, 'wasnt': 171, 'ending': 172, 'place': 173, 'book': 174, 'looks': 175, 'watched': 176, 'three': 177, 'set': 178, 'sense': 179, 'play': 180, 'effects': 181, 'screen': 182, 'reason': 183, 'plays': 184, 'someone': 185, 'true': 186, 'job': 187, 'actor': 188, 'money': 189, 'together': 190, 'said': 191, 'everyone': 192, 'seem': 193, 'later': 194, 'american': 195, 'takes': 196, 'john': 197, 'instead': 198, 'special': 199, 'seeing': 200, 'beautiful': 201, 'idea': 202, 'excellent': 203, 'war': 204, 'audience': 205, 'left': 206, 'shot': 207, 'version': 208, 'night': 209, 'youre': 210, 'high': 211, 'simply': 212, 'black': 213, 'used': 214, 'nice': 215, 'completely': 216, 'read': 217, 'else': 218, 'poor': 219, 'fan': 220, 'kids': 221, 'along': 222, 'house': 223, 'friends': 224, 'help': 225, 'death': 226, 'second': 227, 'short': 228, 'year': 229, 'wife': 230, 'star': 231, 'less': 232, 'try': 233, 'mind': 234, 'given': 235, 'need': 236, 'use': 237, 'enjoy': 238, 'truly': 239, 'classic': 240, 'half': 241, 'boring': 242, 'home': 243, 'either': 244, 'next': 245, 'rest': 246, 'hollywood': 247, 'couple': 248, 'recommend': 249, 'men': 250, 'dead': 251, 'performances': 252, 'tell': 253, 'wrong': 254, 'getting': 255, 'stupid': 256, 'production': 257, 'full': 258, 'remember': 259, 'line': 260, 'start': 261, 'came': 262, 'father': 263, 'let': 264, 'terrible': 265, 'camera': 266, 'understand': 267, 'others': 268, 'school': 269, 'mean': 270, 'perhaps': 271, 'awful': 272, 'keep': 273, 'women': 274, 'wonderful': 275, 'name': 276, 'sex': 277, 'moments': 278, 'definitely': 279, 'playing': 280, 'small': 281, 'top': 282, 'early': 283, 'episode': 284, 'human': 285, 'gives': 286, 'perfect': 287, 'often': 288, 'absolutely': 289, 'budget': 290, 'video': 291, 'person': 292, 'couldnt': 293, 'lines': 294, 'stars': 295, 'went': 296, 'guys': 297, 'finally': 298, 'face': 299, 'head': 300, 'dialogue': 301, 'piece': 302, 'certainly': 303, 'loved': 304, 'case': 305, 'yes': 306, 'liked': 307, 'mr': 308, 'several': 309, 'become': 310, 'title': 311, 'overall': 312, 'hope': 313, 'supposed': 314, 'entire': 315, 'laugh': 316, 'sort': 317, 'written': 318, 'style': 319, 'entertaining': 320, 'felt': 321, 'worse': 322, 'live': 323, 'lost': 324, 'boy': 325, 'mother': 326, 'seemed': 327, 'oh': 328, 'totally': 329, 'problem': 330, 'beginning': 331, 'sound': 332, 'picture': 333, 'friend': 334, 'waste': 335, 'based': 336, 'shes': 337, 'wanted': 338, 'id': 339, 'humor': 340, 'dark': 341, 'fans': 342, 'care': 343, 'final': 344, 'already': 345, 'cinema': 346, 'lead': 347, 'called': 348, 'example': 349, 'despite': 350, 'youll': 351, 'low': 352, 'direction': 353, 'guess': 354, 'girls': 355, 'wont': 356, 'evil': 357, 'becomes': 358, 'throughout': 359, 'able': 360, 'drama': 361, 'turn': 362, 'days': 363, 'unfortunately': 364, 'children': 365, 'wants': 366, 'fine': 367, 'lives': 368, 'white': 369, 'amazing': 370, 'killer': 371, 'game': 372, 'horrible': 373, 'theyre': 374, 'enjoyed': 375, 'quality': 376, 'gave': 377, 'kill': 378, 'act': 379, 'history': 380, 'expect': 381, 'works': 382, 'tries': 383, 'writing': 384, 'favorite': 385, 'michael': 386, 'turns': 387, 'directed': 388, 'parts': 389, 'past': 390, 'run': 391, 'sometimes': 392, 'soon': 393, 'son': 394, 'behind': 395, 'side': 396, 'town': 397, 'flick': 398, 'obviously': 399, 'matter': 400, 'brilliant': 401, 'stuff': 402, 'car': 403, 'ones': 404, 'viewer': 405, 'says': 406, 'thinking': 407, 'starts': 408, 'heard': 409, 'actress': 410, 'highly': 411, 'killed': 412, 'genre': 413, 'decent': 414, 'group': 415, 'eyes': 416, 'heart': 417, 'late': 418, 'took': 419, 'ill': 420, 'happens': 421, 'city': 422, 'kid': 423, 'except': 424, 'art': 425, 'feeling': 426, 'leave': 427, 'wouldnt': 428, 'hell': 429, 'child': 430, 'close': 431, 'extremely': 432, 'police': 433, 'hour': 434, 'coming': 435, 'etc': 436, 'fight': 437, 'blood': 438, 'looked': 439, 'wonder': 440, 'told': 441, 'moment': 442, 'james': 443, 'experience': 444, 'cannot': 445, 'type': 446, 'save': 447, 'cool': 448, 'voice': 449, 'lack': 450, 'score': 451, 'shown': 452, 'including': 453, 'stories': 454, 'ok': 455, 'complete': 456, 'involved': 457, 'stop': 458, 'hilarious': 459, 'taken': 460, 'david': 461, 'chance': 462, 'simple': 463, 'roles': 464, 'obvious': 465, 'murder': 466, 'anyway': 467, 'living': 468, 'god': 469, 'violence': 470, 'lets': 471, 'hand': 472, 'strong': 473, 'itbr': 474, 'happen': 475, 'particularly': 476, 'daughter': 477, 'serious': 478, 'number': 479, 'attempt': 480, 'robert': 481, 'song': 482, 'happened': 483, 'ago': 484, 'opening': 485, 'please': 486, 'jokes': 487, 'released': 488, 'known': 489, 'wish': 490, 'across': 491, 'hit': 492, 'sad': 493, 'usually': 494, 'none': 495, 'slow': 496, 'exactly': 497, 'relationship': 498, 'annoying': 499, 'huge': 500, 'hours': 501, 'career': 502, 'interest': 503, 'english': 504, 'whose': 505, 'reality': 506, 'seriously': 507, 'today': 508, 'body': 509, 'cut': 510, 'saying': 511, 'major': 512, 'cinematography': 513, 'crap': 514, 'alone': 515, 'call': 516, 'order': 517, 'mostly': 518, 'gore': 519, 'started': 520, 'talent': 521, 'usual': 522, 'hero': 523, 'possible': 524, 'age': 525, 'shots': 526, 'female': 527, 'knew': 528, 'running': 529, 'somewhat': 530, 'ridiculous': 531, 'rating': 532, 'brother': 533, 'ends': 534, 'taking': 535, 'important': 536, 'view': 537, 'words': 538, 'knows': 539, 'change': 540, 'due': 541, 'beyond': 542, 'basically': 543, 'power': 544, 'directors': 545, 'scary': 546, 'finds': 547, 'opinion': 548, 'whats': 549, 'apparently': 550, 'arent': 551, 'silly': 552, 'episodes': 553, 'talking': 554, 'level': 555, 'documentary': 556, 'turned': 557, 'clearly': 558, 'husband': 559, 'country': 560, 'attention': 561, 'jack': 562, 'word': 563, 'musical': 564, 'upon': 565, 'local': 566, 'single': 567, 'british': 568, 'talk': 569, 'television': 570, 'novel': 571, 'disappointed': 572, 'room': 573, 'happy': 574, 'strange': 575, 'moviebr': 576, 'cheap': 577, 'tells': 578, 'problems': 579, 'four': 580, 'miss': 581, 'songs': 582, 'french': 583, 'giving': 584, 'easily': 585, 'sequence': 586, 'events': 587, 'modern': 588, 'light': 589, 'whether': 590, 'supporting': 591, 'havent': 592, 'add': 593, 'appears': 594, 'lots': 595, 'bring': 596, 'predictable': 597, 'dialog': 598, 'review': 599, 'bunch': 600, 'hate': 601, 'falls': 602, 'soundtrack': 603, 'similar': 604, 'filmbr': 605, 'certain': 606, 'showing': 607, 'sets': 608, 'enjoyable': 609, 'parents': 610, 'tried': 611, 'mention': 612, 'fall': 613, 'needs': 614, 'surprised': 615, 'future': 616, 'comic': 617, 'ten': 618, 'romantic': 619, 'within': 620, 'team': 621, 'george': 622, 'space': 623, 'message': 624, 'viewers': 625, 'filmed': 626, 'moving': 627, 'theme': 628, 'five': 629, 'sorry': 630, 'earth': 631, 'entertainment': 632, 'ways': 633, 'thriller': 634, 'stay': 635, 'easy': 636, 'storyline': 637, 'dull': 638, 'middle': 639, 'among': 640, 'clear': 641, 'kept': 642, 'named': 643, 'theater': 644, 'nearly': 645, 'king': 646, 'comments': 647, 'sequel': 648, 'die': 649, 'effort': 650, 'feels': 651, 'working': 652, 'near': 653, 'elements': 654, 'greatest': 655, 'doubt': 656, 'deal': 657, 'brought': 658, 'avoid': 659, 'writer': 660, 'richard': 661, 'editing': 662, 'release': 663, 'rock': 664, 'gone': 665, 'using': 666, 'class': 667, 'okay': 668, 'actual': 669, 'youve': 670, 'forget': 671, 'fantastic': 672, 'dr': 673, 'buy': 674, 'means': 675, 'straight': 676, 'monster': 677, 'feature': 678, 'imagine': 679, 'viewing': 680, 'general': 681, 'peter': 682, 'points': 683, 'tale': 684, 'leads': 685, 'hear': 686, 'typical': 687, 'lady': 688, 'suspense': 689, 'famous': 690, 'reviews': 691, 'move': 692, 'minute': 693, 'sister': 694, 'boys': 695, 'subject': 696, 'check': 697, 'form': 698, 'period': 699, 'realistic': 700, 'begins': 701, 'whos': 702, 'th': 703, 'material': 704, 'rent': 705, 'dog': 706, 'animation': 707, 'particular': 708, 'sit': 709, 'sequences': 710, 'somehow': 711, 'mystery': 712, 'decided': 713, 'tom': 714, 'learn': 715, 'expected': 716, 'killing': 717, 'fast': 718, 'believable': 719, 'eventually': 720, 'paul': 721, 'atmosphere': 722, 'became': 723, 'eye': 724, 'york': 725, 'premise': 726, 'difficult': 727, 'red': 728, 'poorly': 729, 'surprise': 730, 'average': 731, 'indeed': 732, 'possibly': 733, 'weak': 734, 'crime': 735, 'doctor': 736, 'needed': 737, 'figure': 738, 'leaves': 739, 'lame': 740, 'whatever': 741, 'crew': 742, 'deep': 743, 'emotional': 744, 'follow': 745, 'stand': 746, 'scifi': 747, 'reading': 748, 'romance': 749, 'open': 750, 'footage': 751, 'wait': 752, 'truth': 753, 'question': 754, 'situation': 755, 'dance': 756, 'memorable': 757, 'meet': 758, 'credits': 759, 'otherwise': 760, 'begin': 761, 'forced': 762, 'unless': 763, 'shame': 764, 'western': 765, 'writers': 766, 'previous': 767, 'note': 768, 'sexual': 769, 'write': 770, 'gay': 771, 'cheesy': 772, 'imdb': 773, 'sounds': 774, 'keeps': 775, 'third': 776, 'oscar': 777, 'joe': 778, 'nature': 779, 'de': 780, 'features': 781, 'towards': 782, 'interested': 783, 'hands': 784, 'personal': 785, 'hot': 786, 'total': 787, 'unique': 788, 'island': 789, 'quickly': 790, 'filmmakers': 791, 'male': 792, 'meets': 793, 'weird': 794, 'superb': 795, 'earlier': 796, 'inside': 797, 'copy': 798, 'plus': 799, 'season': 800, 'battle': 801, 'crazy': 802, 'masterpiece': 803, 'beauty': 804, 'b': 805, 'screenplay': 806, 'box': 807, 'badly': 808, 'worked': 809, 'effect': 810, 'japanese': 811, 'incredibly': 812, 'result': 813, 'realize': 814, 'admit': 815, 'setting': 816, 'comment': 817, 'laughs': 818, 'stage': 819, 'air': 820, 'background': 821, 'street': 822, 'plenty': 823, 'brings': 824, 'older': 825, 'following': 826, 'fairly': 827, 'powerful': 828, 'society': 829, 'bill': 830, 'free': 831, 'perfectly': 832, 'leading': 833, 'various': 834, 'creepy': 835, 'mark': 836, 'baby': 837, 'appear': 838, 'lee': 839, 'business': 840, 'acted': 841, 'america': 842, 'forward': 843, 'development': 844, 'dramatic': 845, 'william': 846, 'ask': 847, 'deserves': 848, 'manages': 849, 'expecting': 850, 'caught': 851, 'meant': 852, 'outside': 853, 'ideas': 854, 'reasons': 855, 'pay': 856, 'apart': 857, 'twist': 858, 'directing': 859, 'zombie': 860, 'recently': 861, 'spent': 862, 'portrayed': 863, 'dumb': 864, 'brothers': 865, 'political': 866, 'remake': 867, 'wasted': 868, 'attempts': 869, 'dream': 870, 'agree': 871, 'success': 872, 'water': 873, 'missing': 874, 'mess': 875, 'fails': 876, 'german': 877, 'front': 878, 'members': 879, 'cover': 880, 'fire': 881, 'large': 882, 'odd': 883, 'return': 884, 'unlike': 885, 'rich': 886, 'joke': 887, 'disney': 888, 'present': 889, 'secret': 890, 'girlfriend': 891, 'waiting': 892, 'hardly': 893, 'la': 894, 'van': 895, 'incredible': 896, 'cute': 897, 'casting': 898, 'plain': 899, 'escape': 900, 'cop': 901, 'talented': 902, 'party': 903, 'create': 904, 'era': 905, 'match': 906, 'public': 907, 'clever': 908, 'laughing': 909, 'fighting': 910, 'pace': 911, 'created': 912, 'visual': 913, 'gun': 914, 'cause': 915, 'villain': 916, 'potential': 917, 'married': 918, 'italian': 919, 'popular': 920, 'telling': 921, 'sadly': 922, 'died': 923, 'hold': 924, 'state': 925, 'considering': 926, 'bored': 927, 'break': 928, 'fantasy': 929, 'ended': 930, 'nudity': 931, 'familiar': 932, 'pure': 933, 'list': 934, 'entirely': 935, 'follows': 936, 'uses': 937, 'cartoon': 938, 'sees': 939, 'biggest': 940, 'rate': 941, 'flat': 942, 'missed': 943, 'wrote': 944, 'decides': 945, 'slightly': 946, 'language': 947, 'tension': 948, 'former': 949, 'office': 950, 'compared': 951, 'fit': 952, 'images': 953, 'appreciate': 954, 'mentioned': 955, 'speak': 956, 'company': 957, 'credit': 958, 'portrayal': 959, 'kills': 960, 'younger': 961, 'trouble': 962, 'produced': 963, 'suddenly': 964, 'convincing': 965, 'producers': 966, 'moves': 967, 'amount': 968, 'recent': 969, 'audiences': 970, 'sweet': 971, 'neither': 972, 'successful': 973, 'college': 974, 'band': 975, 'violent': 976, 'filled': 977, 'involving': 978, 'cold': 979, 'value': 980, 'train': 981, 'force': 982, 'choice': 983, 'recommended': 984, 'amusing': 985, 'ultimately': 986, 'intelligent': 987, 'consider': 988, 'science': 989, 'store': 990, 'dancing': 991, 'fear': 992, 'control': 993, 'mad': 994, 'depth': 995, 'pointless': 996, 'exciting': 997, 'christmas': 998, 'charles': 999, 'master': 1000}


In [14]:
class SentimentRNN_bi(nn.Module):
    def __init__(self, no_layers, vocab_size, hidden_dim, embedding_dim, output_dim, drop_prob=0.5):
        super(SentimentRNN_bi, self).__init__()

        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.no_layers = no_layers
        self.vocab_size = vocab_size

        # embedding and bi-directional LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # bi-directional lstm
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=self.hidden_dim,
                            num_layers=no_layers, batch_first=True, bidirectional=True)

        # dropout layer
        self.dropout = nn.Dropout(0.3)

        # linear and sigmoid layer
        # Since the LSTM is bidirectional, the hidden size is doubled
        self.fc = nn.Linear(2 * self.hidden_dim, output_dim)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden):
        batch_size = x.size(0)
        # embeddings and lstm_out
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)

        # concatenate the forward and backward hidden states
        lstm_out = torch.cat((hidden[0][-2, :, :], hidden[0][-1, :, :]), dim=1)

        # dropout and fully connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # sigmoid function
        sig_out = self.sig(out)

        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)

        sig_out = sig_out[:, -1]  # get last batch of labels

        # return last sigmoid output and hidden state
        return sig_out, hidden

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes 2 * n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of bidirectional LSTM
        h0 = torch.zeros((2 * self.no_layers, batch_size, self.hidden_dim)).to(device)
        c0 = torch.zeros((2 * self.no_layers, batch_size, self.hidden_dim)).to(device)
        hidden = (h0, c0)
        return hidden

In [19]:
def preprocess_string(s):
    # Remove all non-word characters (everything except numbers and letters)
    s = re.sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespaces with no space
    s = re.sub(r"\s+", '', s)
    # replace digits with no space
    s = re.sub(r"\d", '', s)

    return s
def padding_(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

# model = torch.load('NLP_model.pt')
# model.eval() 
no_layers = 2
vocab_size = len(vocab) + 1 #extra 1 for padding
embedding_dim = 64
output_dim = 1
hidden_dim = 256


model = SentimentRNN_bi(no_layers,vocab_size,hidden_dim,embedding_dim,output_dim,drop_prob=0.5)
model.load_state_dict(torch.load('NLP_model.pt', map_location=torch.device('cpu')))
model.eval()

def predict_text(text):
        word_seq = np.array([vocab[preprocess_string(word)] for word in text.split() 
                         if preprocess_string(word) in vocab.keys()])
        word_seq = np.expand_dims(word_seq,axis=0)
        pad =  torch.from_numpy(padding_(word_seq,500))
        inputs = pad.to(device)
        batch_size = 1
        h = model.init_hidden(batch_size)
        h = tuple([each.data for each in h])
        output, h = model(inputs, h)
        return(output.item())


In [20]:

app = FastAPI()

class Text(BaseModel):
    text: str

@app.post("/predict")
async def predict_sentiment(text: Text):
    pro = predict_text(text.text)
    status = "positive" if pro > 0.5 else "negative"
    pro = (1 - pro) if status == "negative" else pro
    print(f'Predicted sentiment is {status} with a probability of {pro}')
    return {"prediction": status, "probability": pro}

In [25]:
from flask import Flask, request, jsonify, render_template

app = Flask(__name__)

@app.route("/", methods=["GET"])
def home():
    return render_template('sentiment_web.html')  # replace 'your_html_file.html' with your actual HTML file name

@app.route("/predict", methods=["POST"])
def predict_sentiment():
    text = request.json['text']
    pro = predict_text(text)
    status = "positive" if pro > 0.5 else "negative"
    pro = (1 - pro) if status == "negative" else pro
    print(f'Predicted sentiment is {status} with a probability of {pro}')
    return jsonify({"prediction": status, "probability": pro})

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/shashank/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/shashank/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/shashank/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/home/shashank/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/home/shashank/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets


SystemExit: 1